<a href="https://colab.research.google.com/github/danicrg/whatsapp-analyser/blob/master/whatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whatsapp Chat Analyser

In [ ]:
!pip install emoji

In [ ]:
import re
import regex
import pandas as pd
import numpy as np
import emoji

## Parsing The Chat

In [ ]:
f = open('chat.txt', 'r')

In [ ]:
line = f.readline().strip()

In [ ]:
def startsWithDateAndTimeAndroid(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

def startsWithDateAndTimeios(s):
    pattern = '^\[([0-9]+)([\/-])([0-9]+)([\/-])([0-9]+)[,]? ([0-9]+):([0-9][0-9]):([0-9][0-9])?[ ]?(AM|PM|am|pm)?\]' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
def findAuthor(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

In [ ]:
def getDataPointAndroid(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if findAuthor(message): 
        splitMessage = message.split(':') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message

def getDataPointios(line):
	splitLine = line.split('] ')
	dateTime = splitLine[0]
	if ',' in dateTime:
		date, time = dateTime.split(',')
	else:
		date, time = dateTime.split(' ')
	message = ' '.join(splitLine[1:])
	if findAuthor(message):
		splitMessage = message.split(':')
		author = splitMessage[0]
		message = ' '.join(splitMessage[1:])
	else:
		author = None
	if time[5]==":":
		time = time[:5]+time[-3:]
	else:
		if 'AM' in time or 'PM' in time:
			time = time[:6]+time[-3:]
		else:
			time = time[:6]
	return date, time, author, message

In [ ]:
def dateconv(date):
  year=''
  if '-' in date:
    year = date.split('-')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d-%m-%Y").strftime("%Y-%m-%d")
    elif len(year) ==2:
      return datetime.datetime.strptime(date, "[%d-%m-%y").strftime("%Y-%m-%d")
  elif '/' in date:
    year = date.split('/')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d/%m/%Y").strftime("%Y-%m-%d")
    if len(year) ==2:
      return datetime.datetime.strptime(date, "[%d/%m/%y").strftime("%Y-%m-%d")

In [ ]:
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [ ]:
parsedData = []
conversationPath = 'chat.txt' 
with open(conversationPath, encoding="utf-8") as fp:
    device=''
    first=fp.readline()
    if '[' in first:
      device='ios'
    else:
      device="android"
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        if device=="ios":
          line = line.strip()
          if startsWithDateAndTimeios(line):
            if len(messageBuffer) > 0:
              parsedData.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDataPointios(line)
            messageBuffer.append(message)
          else:
            line= (line.encode('ascii', 'ignore')).decode("utf-8")
            if startsWithDateAndTimeios(line):
              if len(messageBuffer) > 0:
                parsedData.append([date, time, author, ' '.join(messageBuffer)])
              messageBuffer.clear()
              date, time, author, message = getDataPointios(line)
              messageBuffer.append(message)
            else:
              messageBuffer.append(line)
        else:
          line = line.strip()
          if startsWithDateAndTimeAndroid(line):
            if len(messageBuffer) > 0:
              parsedData.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDataPointAndroid(line)
            messageBuffer.append(message)
          else:
            messageBuffer.append(line)

7/16/20, 10:35 AM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.



In [ ]:
if device =='android':
        df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.dropna()
        df["emoji"] = df["Message"].apply(split_count)
        URLPATTERN = r'(https?://\S+)'
        df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
else:
        df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
        df = df.dropna()
        df["Date"] = df["Date"].apply(dateconv)
        df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
        df["emoji"] = df["Message"].apply(split_count)
        URLPATTERN = r'(https?://\S+)'
        df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()

## Number of messages

In [ ]:
number_of_messages = df.groupby('Author')['Message'].count()
number_of_messages

Author
Daniel Carlander    2456
Isa Menéndez        2903
Name: Message, dtype: int64

## Number of words

It includes emojis as words

In [ ]:
df_no_media = df[~df['Message'].str.contains('<Media omitted>')]

df_no_media['MessageLength'] = df_no_media['Message'].str.split().str.len()

In [ ]:
number_of_words = df_no_media.groupby('Author')['MessageLength'].sum()
number_of_words

Author
Daniel Carlander    12004
Isa Menéndez        12692
Name: MessageLength, dtype: int64

## Average words per message

In [ ]:
number_of_words / df_no_media.groupby('Author')['Message'].count()

Author
Daniel Carlander    5.160791
Isa Menéndez        4.595221
dtype: float64

In [ ]:
df

,Date,Time,Author,Message,emoji,urlcount,Message length
0,2020-07-16,10:36 AM,Isa Menéndez,Jajajaajaj oye que a veces toca 😂,[😂],0,7
1,2020-07-16,10:36 AM,Isa Menéndez,Nah es que tengo dermatólogo que me fui a dor...,[😵],0,15
2,2020-07-16,10:42 AM,Daniel Carlander,Buenas copas entonces jaajaj,[],0,4
3,2020-07-16,10:44 AM,Daniel Carlander,A aprovechar el día entonces 😉,[😉],0,6
4,2020-07-16,10:50 AM,Isa Menéndez,Jajajaja hombre es que hacía que no veía a mi...,[],0,15
...,...,...,...,...,...,...,...
5374,2020-09-21,9:48 AM,Daniel Carlander,<Media omitted>,[],0,2
5375,2020-09-21,9:48 AM,Daniel Carlander,También es valiidd,[],0,3
5376,2020-09-21,9:55 AM,Daniel Carlander,Que vaya genial la prueba!! A reventarlo 💪🏼,[💪🏼],0,8
5377,2020-09-21,9:59 AM,Isa Menéndez,Jajajajaja exactooo,[],0,2
